In [10]:
from keras.applications import MobileNet
from keras.layers import Dense,Dropout,GlobalAveragePooling2D,Conv2D,MaxPooling2D,Activation

rows,columns = 224,224

mobilenet = MobileNet(weights='imagenet',
                     include_top=False,
                     input_shape=(rows,columns,3))
for layer in mobilenet.layers:
    layer.trainable = False

for i,layers in enumerate(mobilenet.layers):
    print('Layer {} --> '.format(i+1),layers.__class__.__name__,layers.trainable)

Layer 1 -->  InputLayer False
Layer 2 -->  ZeroPadding2D False
Layer 3 -->  Conv2D False
Layer 4 -->  BatchNormalization False
Layer 5 -->  ReLU False
Layer 6 -->  DepthwiseConv2D False
Layer 7 -->  BatchNormalization False
Layer 8 -->  ReLU False
Layer 9 -->  Conv2D False
Layer 10 -->  BatchNormalization False
Layer 11 -->  ReLU False
Layer 12 -->  ZeroPadding2D False
Layer 13 -->  DepthwiseConv2D False
Layer 14 -->  BatchNormalization False
Layer 15 -->  ReLU False
Layer 16 -->  Conv2D False
Layer 17 -->  BatchNormalization False
Layer 18 -->  ReLU False
Layer 19 -->  DepthwiseConv2D False
Layer 20 -->  BatchNormalization False
Layer 21 -->  ReLU False
Layer 22 -->  Conv2D False
Layer 23 -->  BatchNormalization False
Layer 24 -->  ReLU False
Layer 25 -->  ZeroPadding2D False
Layer 26 -->  DepthwiseConv2D False
Layer 27 -->  BatchNormalization False
Layer 28 -->  ReLU False
Layer 29 -->  Conv2D False
Layer 30 -->  BatchNormalization False
Layer 31 -->  ReLU False
Layer 32 -->  Depthwi

In [16]:
# Creates the head of the model that will be placed upon the bottom model
# we r saying it head but it is not actually the head as it is being joined after the 87 layers as we saw above 
# it is actually the output side
def add_top_layers_to_MobileNet(bottom_model,num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [17]:
from keras.models import Model

self_made_layers = add_top_layers_to_MobileNet(mobilenet,10)
model = Model(inputs = mobilenet.input , outputs = self_made_layers)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [19]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = 'monkey_breed\\train' 
validation_dir = 'monkey_breed\\validation'

train_gen = ImageDataGenerator(rescale=1./255,
                              rotation_range=45,
                              height_shift_range=0.3,
                              width_shift_range=0.3,
                              horizontal_flip=True,
                              fill_mode='nearest')

validation_gen = ImageDataGenerator(rescale=1./255)

batch_size = 32

training_data_generator = train_gen.flow_from_directory(train_dir,
                                                       batch_size=batch_size,
                                                       shuffle=True,
                                                       class_mode='categorical',
                                                       target_size=(rows,columns))

valiadtion_data_generator = validation_gen.flow_from_directory(validation_dir,
                                                       batch_size=batch_size,
                                                       shuffle=False,
                                                       class_mode='categorical',
                                                       target_size=(rows,columns))


Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [21]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.optimizers import RMSprop

checkpoint = ModelCheckpoint('model/monkey_breed_MobileNet_V1.h5',
                             verbose=1,
                             mode='min',
                             monitor='val_loss',
                             save_best_only=True)

earlystopping = EarlyStopping(verbose=1,
                              monitor='val_loss',
                              restore_best_weights=True,
                              patience=3,
                              min_delta=0)

callbacks = [earlystopping,checkpoint] 

model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(learning_rate=0.001),
             metrics=['accuracy'])


train_samples = 1098
validation_samples = 272

epochs = 5
batch_size = 16

history = model.fit_generator(training_data_generator,
                              epochs=epochs,
                              steps_per_epoch=train_samples//batch_size,
                              validation_data=valiadtion_data_generator,
                              callbacks=callbacks,
                              validation_steps=validation_samples//batch_size)



W1114 12:41:48.636583 14688 deprecation_wrapper.py:119] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/5
68/68 [==============================] - 270s 4s/step - loss: 1.7722 - accuracy: 0.4883 - val_loss: 0.8343 - val_accuracy: 0.7027

Epoch 00001: val_loss improved from inf to 0.83427, saving model to model/monkey_breed_MobileNet_V1.h5
Epoch 2/5
68/68 [==============================] - 258s 4s/step - loss: 0.7730 - accuracy: 0.7772 - val_loss: 0.3949 - val_accuracy: 0.9121

Epoch 00002: val_loss improved from 0.83427 to 0.39487, saving model to model/monkey_breed_MobileNet_V1.h5
Epoch 3/5
68/68 [==============================] - 265s 4s/step - loss: 0.6078 - accuracy: 0.8293 - val_loss: 0.1868 - val_accuracy: 0.8047

Epoch 00003: val_loss improved from 0.39487 to 0.18679, saving model to model/monkey_breed_MobileNet_V1.h5
Epoch 4/5
68/68 [==============================] - 261s 4s/step - loss: 0.4509 - accuracy: 0.8691 - val_loss: 0.0588 - val_accuracy: 0.9297

Epoch 00004: val_loss improved from 0.18679 to 0.05881, saving model to model/monkey_breed_MobileNet_V1.h5
Epoch 5/5
68

In [52]:
import os
import cv2
import numpy as np
import random
from keras.models import load_model

model = load_model('model\\monkey_breed_MobileNet_V1.h5')
acc=[]
monkey_breed_dict={'[0]':'mantled_howler',
                   '[1]':'patas_monkey',
                   '[2]':'bald_uakari',
                   '[3]':'japanese_macaque',
                   '[4]':'pygmy_marmoset',
                   '[5]':'white_headed_capuchin',
                   '[6]':'silvery_marmoset',
                   '[7]':'common_squirrel_monkey',
                   '[8]':'black_headed_night_monkey',
                   '[9]':'nilgiri_langur'}

for i in range(10):
    monkey_class = np.random.randint(0,9)
    all_class_list    = os.listdir('monkey_breed\\validation')
    select_one_class = all_class_list[monkey_class] 
    pick_one_pic = random.choice(os.listdir('monkey_breed\\validation\\'+str(select_one_class)+'\\'))
    org_img = cv2.imread('monkey_breed\\validation\\'+str(select_one_class)+'\\'+pick_one_pic)
    org_img = cv2.resize(org_img,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
    img=org_img.copy()
    cv2.imshow('random img',img)
    img = cv2.resize(img,(224,224),interpolation=cv2.INTER_LINEAR)
    img = img/255
    img = img.reshape(1,224,224,3)
    print('true --> ',monkey_breed_dict[str([monkey_class])])
    print('predicted -->',monkey_breed_dict[str([np.argmax(model.predict(img,1,verbose=0))])])
    acc.append(np.equal(monkey_class,np.argmax(model.predict(img,1,verbose=0))))
    cv2.waitKey(0)
cv2.destroyAllWindows()
print('\n')
print('Accuracy ---> ',str(round(np.mean(acc)*100,2))+' %')

true -->  japanese_macaque
predicted --> japanese_macaque
true -->  common_squirrel_monkey
predicted --> common_squirrel_monkey
true -->  mantled_howler
predicted --> mantled_howler
true -->  silvery_marmoset
predicted --> white_headed_capuchin
true -->  mantled_howler
predicted --> mantled_howler
true -->  bald_uakari
predicted --> nilgiri_langur
true -->  bald_uakari
predicted --> bald_uakari
true -->  common_squirrel_monkey
predicted --> common_squirrel_monkey
true -->  white_headed_capuchin
predicted --> white_headed_capuchin
true -->  japanese_macaque
predicted --> japanese_macaque


Accuracy --->  80.0 %
